https://openmdao.org/newdocs/versions/latest/examples/hohmann_transfer/hohmann_transfer.html
https://en.wikipedia.org/wiki/Hohmann_transfer_orbit

In [40]:
from cosapp.systems import System
from cosapp.drivers import Optimizer

import numpy as np

In [49]:
class Hohmann(System):

    def setup(self):
        self.add_inward('mu', 398600.4418, desc='Gravitational parameter of central body', unit='km**3/s**2')

        self.add_inward('rp', 7000.0, desc='periapsis radius', unit='km')
        self.add_inward('ra', 42164.0, desc='apoapsis radius', unit='km')
        self.add_inward('incp', 28.5, unit='deg',desc='periapsis inclinaison')
        self.add_inward('inca', 0., unit='deg', desc='apoapsis inclinaison')

        self.add_inward('inc', self.incp, unit='deg', desc='GTO inclinaison')

        self.add_outward('delta_v', 0., unit='km/s')
        self.add_outward('delta_inc', 0., unit='deg')

        self.add_outward('vp', 0., unit='km/s')
        self.add_outward('va', 0., unit='km/s')
        self.add_outward('vp_transfer', 0., unit='km/s')
        self.add_outward('va_transfer', 0., unit='km/s')
        
        # design method
        self.add_design_method('gto_inc').add_unknown('inc')

        
    def compute(self):
        def vCircComp(r, mu=398600.4418) :
            """
            Computes the circular orbit velocity given a radius and gravitational
            parameter.
            """
            return np.sqrt(mu / r)

        def transferOrbitComp(rp, ra, mu=398600.4418) -> (float, float):
            a = (ra + rp) / 2.0
            e = (a - rp) / a
            p = a * (1.0 - e ** 2)
            h = np.sqrt(mu * p)

            vp = h / rp
            va = h / ra
            return (vp, va)

        def delta_vComp(v1, v2, dinc) -> float:
            """
            Compute the delta-V performed given the magnitude of two velocities
            and the angle between them.
            """
            return np.sqrt(v1 ** 2 + v2 ** 2 - 2.0 * v1 * v2 * np.cos(dinc * np.pi / 180.))        
        
        mu = self.mu
        
        rp = self.rp
        ra = self.ra
        
        self.vp = vp = vCircComp(rp, mu)
        self.va = va = vCircComp(ra, mu)
                
        (self.vp_transfer, self.va_transfer) = (vp_transfer, va_transfer) = transferOrbitComp(rp, ra, mu)

        inc = self.inc
        inca = self.inca
        incp = self.incp

        delta_v1 = delta_vComp(vp, vp_transfer, inc - incp)
        delta_v2 = delta_vComp(va_transfer, va, inca - inc)
        
        self.delta_v = delta_v1 + delta_v2
        self.delta_inc = inca - incp
                        

In [55]:
# create System
prob = Hohmann('hohmann')
prob.rp = 6678
prob.ra = 42164

prob.incp = 28.5
prob.inc = 0

prob.run_drivers()
assert abs(prob.vp - 7.73) < 1e-2
assert abs(prob.va - 3.07) < 1e-2

print('from vp to vp_transfer : ', prob.vp, prob.vp_transfer)
print('from va_transfer to va : ', prob.va_transfer, prob.va)

print('non optimal delta_v ', prob.delta_v)

# setup the optimization

opt = prob.add_driver(Optimizer('optimization'))

opt.runner.design.extend(prob.design_methods['gto_inc'])
opt.runner.set_objective('delta_v')

# run
prob.run_drivers()

print('optimal delta_v ', prob.delta_v)
print('optimal delta_inc ', prob.incp - prob.inc)

from vp to vp_transfer :  7.72583947913639 10.151608507443248
from va_transfer to va :  1.6078275688432315 3.074666284127684
non optimal delta_v  6.456130232084301
optimal delta_v  4.2313545928422815
optimal delta_inc  2.2001563573042695


Driver 'optimization' already exists and will be replaced


-28.5
4.267658111781678
2.1639629765204624


In [34]:
prob.inputs

OrderedDict([('inwards',
              ExtensiblePort: {'mu': 398600.4418, 'rp': 6578, 'ra': 42270, 'incp': 28.5, 'inca': 0.0, 'inc': 26.336036978479534})])